<a href="https://colab.research.google.com/github/Vita8759/4_Layers/blob/main/WL_dif(400)v1_0_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision import datasets, transforms

In [ ]:
CUDA = torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")
print(device)

cpu


In [ ]:
import json
import zipfile
import os

# kaggle api
api_token = {"username":"pinhsuanlee","key":"28c330210349e9af127aaf6f88dfa39f"}
 
if not os.path.exists("/root/.kaggle"):
    os.makedirs("/root/.kaggle")

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json
 
if not os.path.exists("/kaggle"):
    os.makedirs("/kaggle")
os.chdir('/kaggle')
!kaggle datasets download -d pinhsuanlee/wl-difv1 --force
 
!ls /kaggle

 85% 11.0M/12.9M [00:00<00:00, 115MB/s]
100% 12.9M/12.9M [00:00<00:00, 128MB/s]
wl-difv1.zip


In [ ]:
!unzip wl-difv1.zip

Archive:  wl-difv1.zip
  inflating: dataset/test_set/bad/bad(33).png  
  inflating: dataset/test_set/bad/bad(34).png  
  inflating: dataset/test_set/bad/bad(35).png  
  inflating: dataset/test_set/bad/bad(36).png  
  inflating: dataset/test_set/bad/bad(37).png  
  inflating: dataset/test_set/bad/bad(38).png  
  inflating: dataset/test_set/bad/bad(39).png  
  inflating: dataset/test_set/bad/bad(40).png  
  inflating: dataset/test_set/good/good(11).png  
  inflating: dataset/test_set/good/good(12).png  
  inflating: dataset/training_set/bad/bad(1).png  
  inflating: dataset/training_set/bad/bad(10).png  
  inflating: dataset/training_set/bad/bad(11).png  
  inflating: dataset/training_set/bad/bad(12).png  
  inflating: dataset/training_set/bad/bad(13).png  
  inflating: dataset/training_set/bad/bad(14).png  
  inflating: dataset/training_set/bad/bad(15).png  
  inflating: dataset/training_set/bad/bad(16).png  
  inflating: dataset/training_set/bad/bad(17).png  
  inflating: dataset/train

In [ ]:
# define the neural net class
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=20, 
                               kernel_size=13, stride=4)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=30, 
                               kernel_size=13, stride=4)
        self.fc1 = nn.Linear(in_features=2430, out_features=1000)
        self.fc2 = nn.Linear(in_features=1000, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = Net().to(device)
print(net)

Net(
  (conv1): Conv2d(3, 20, kernel_size=(13, 13), stride=(4, 4))
  (conv2): Conv2d(20, 30, kernel_size=(13, 13), stride=(4, 4))
  (fc1): Linear(in_features=2430, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=10, bias=True)
)


In [ ]:
# loss
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
batch_size = 1
num_epochs = 3
lr = 0.001
num_classes=2

In [ ]:
# Transform
transform = transforms.Compose(
                [transforms.Resize(size=(700,700)),
                 transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,)),]
                )

# Data
train_dataset = datasets.ImageFolder(root='/kaggle/dataset/training_set', transform=transform)
valid_dataset = datasets.ImageFolder(root='/kaggle/dataset/test_set', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import time
start = time.time()

def train(net, train_loader):

    for epoch in range(400): # no. of epochs
        running_loss_1 = 0
        for data in train_loader:
            # data pixels and labels to GPU if available
            inputs, labels = data[0].to(device, non_blocking=True), data[1].to(device, non_blocking=True)
            # set the parameter gradients to zero
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            # propagate the loss backward
            loss.backward()
            # update the gradients
            optimizer.step()
 
            running_loss_1= loss.item()
        print('[Epoch %d] train_loss: %.3f' %
                      (epoch + 1, running_loss_1/len(train_loader)))

    running_loss_1 = []
    epoch = []        


    train_correct = 0
    train_total = 0
    with torch.no_grad():
        for data in train_loader:
            inputs, labels = data[0].to(device, non_blocking=True), data[1].to(device, non_blocking=True)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            print(predicted)


    print('Accuracy of the network on train images: %0.4f %%' % (
        100 * train_correct / train_total))




   
def test(net, valid_loader):
      for epoch in range(400): # no. of epochs
        running_loss_2 = 0
        for data in train_loader:
            # data pixels and labels to GPU if available
            inputs, labels = data[0].to(device, non_blocking=True), data[1].to(device, non_blocking=True)
            # set the parameter gradients to zero
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            # propagate the loss backward
            loss.backward()
            # update the gradients
            optimizer.step()
 
            running_loss_2= loss.item()
        print('[Epoch %d] test_loss: %.3f' %
                      (epoch + 1, running_loss_2/len(valid_loader)))
        
      running_loss_2 = []
      epoch = [] 
        


      test_correct = 0
      test_total = 0
      with torch.no_grad():
        for data in valid_loader:
            inputs, labels = data[0].to(device, non_blocking=True), data[1].to(device, non_blocking=True)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
            print(predicted)


      print('Accuracy of the network on test images: %0.4f %%' % (
        100 * test_correct / test_total))
       
 


      running_loss = []
      epoch = [] 

    

train(net, train_loader)
test(net, valid_loader)       

end = time.time()
print('Import Time :', end - start)

[Epoch 1] train_loss: 0.046
[Epoch 2] train_loss: 0.012
[Epoch 3] train_loss: 0.001
[Epoch 4] train_loss: 0.005
[Epoch 5] train_loss: 0.034
[Epoch 6] train_loss: 0.002
[Epoch 7] train_loss: 0.011
[Epoch 8] train_loss: 0.033
[Epoch 9] train_loss: 0.012
[Epoch 10] train_loss: 0.004
[Epoch 11] train_loss: 0.010
[Epoch 12] train_loss: 0.025
[Epoch 13] train_loss: 0.005
[Epoch 14] train_loss: 0.008
[Epoch 15] train_loss: 0.001
[Epoch 16] train_loss: 0.001
[Epoch 17] train_loss: 0.000
[Epoch 18] train_loss: 0.001
[Epoch 19] train_loss: 0.000
[Epoch 20] train_loss: 0.001
[Epoch 21] train_loss: 0.000
[Epoch 22] train_loss: 0.000
[Epoch 23] train_loss: 0.000
[Epoch 24] train_loss: 0.000
[Epoch 25] train_loss: 0.000
[Epoch 26] train_loss: 0.000
[Epoch 27] train_loss: 0.000
[Epoch 28] train_loss: 0.000
[Epoch 29] train_loss: 0.000
[Epoch 30] train_loss: 0.000
[Epoch 31] train_loss: 0.000
[Epoch 32] train_loss: 0.000
[Epoch 33] train_loss: 0.000
[Epoch 34] train_loss: 0.000
[Epoch 35] train_loss: 